https://rare-technologies.com/word2vec-tutorial/

lo que hacemos es utilizar una red neuronal para interrelacionar las palabras de un texto.
Como?

usando un autoencoder al que luego quitamos la ultima capa, esto nos da los vectores de cada palabra una red neuronal con n entradas (palabras unicas en el texto), m neuronas en capa intermedia, y o neuronas en la salida. el sistema intenta predecir qué palabra vendrá despues de cada palabra de entrada. siendo la palabra un vector de 1Xn.

el sistema aprenderá que palabras con vectores parecidos estan inter relacionadas (pertenecen al mismo tema)

al cortar la capa final, lo que nos queda es una matriz de datos con la probabilidad de aparicion de una palabra para las demas. eso es un vector

In [1]:
import gensim
import pandas as pd
import pyodbc 
import configLocal

In [2]:
server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

In [3]:
data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [4]:
data.head()

,msg_id,msg_users_id,msg_timestamp,msg_text
0,1123283650638942211,428308122,1556646666570,@aritzeta @bibiloni @JosepMClD @Ratafia_Party ...
1,1123283654740983808,1669818008,1556646667548,Que fuerte!!! Y que lastima
2,1123283664123703296,2295120738,1556646669785,"@Helga_tito Bro no me quejo, si me parece de p..."
3,1123283666682220544,3327214991,1556646670395,@iescolar Evidente!!!
4,1123283688136032256,141861159,1556646675510,Karen me manda msj todos los días ??


In [5]:
data.shape

(25586, 4)

In [58]:
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE))
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE)]
type(corpus), len(corpus)

(dict, 5054)

In [46]:
corpus["428308122"]

['@aritzeta @bibiloni @JosepMClD @Ratafia_Party @jbadia16 Jo també. Però diria que el reflexiu de ESTAR és sobrer: \n',
 '@CUPRiudoms Sí.\nHo he vist de casualitat. Millor un MD.',
 '@Josefina170541 @martonamg @fananwa "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@literaturaimes @EnricGoma @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@mauletgroc @martonamg "Quan fou mort el combregaren" equival a "A buenas horas mangas verdes".',
 '@TrudiGonzlezMa1 @martonamg Potser sí.',
 '@victoracedoma @bibiloni @JosepMClD @Ratafia_Party @jbadia16 No, no ho és: ',
 '@Fakerragut Jo no.',
 '@Romeucarles @VogelfreiCAT Sí que ho és.']

In [59]:
for i in corpus.keys():
    corpus[i] = gensim.utils.simple_preprocess("".join(corpus[i]).encode('utf-8'), deacc=True, min_len=3)

In [61]:
corpus["428308122"]

['aritzeta',
 'bibiloni',
 'josepmcld',
 'ratafia_party',
 'jbadia',
 'tambe',
 'pero',
 'diria',
 'que',
 'reflexiu',
 'estar',
 'sobrer',
 'cupriudoms',
 'vist',
 'casualitat',
 'millor',
 'josefina',
 'martonamg',
 'fananwa',
 'quan',
 'fou',
 'mort',
 'combregaren',
 'equival',
 'buenas',
 'horas',
 'mangas',
 'verdes',
 'literaturaimes',
 'enricgoma',
 'martonamg',
 'quan',
 'fou',
 'mort',
 'combregaren',
 'equival',
 'buenas',
 'horas',
 'mangas',
 'verdes',
 'mauletgroc',
 'martonamg',
 'quan',
 'fou',
 'mort',
 'combregaren',
 'equival',
 'buenas',
 'horas',
 'mangas',
 'verdes',
 'trudigonzlezma',
 'martonamg',
 'potser',
 'victoracedoma',
 'bibiloni',
 'josepmcld',
 'ratafia_party',
 'jbadia',
 'fakerragut',
 'romeucarles',
 'vogelfreicat',
 'que']

In [70]:
#prueba de una lista de listas de strings por usuario
df = []
for i in corpus.keys():
    df.append(corpus[i])

In [62]:
df = []
for i in corpus.keys():
    for j in corpus[i]:
        df.append(j)

5054

In [73]:
print("List of lists. Let's confirm: ", type(df), " of ", type(df[0]))

List of lists. Let's confirm:  <class 'list'>  of  <class 'list'>


In [80]:
#size: Nª of layers
#window: Maximum distance between the current and predicted word within a sentence
#min_count: nº of times a word repeats to be taken into account
#sg: training algorithm. 0=cbow, 1=skip-gram
#workers: nº of cores running. no cyton installed--> no paralelization

model = gensim.models.Word2Vec(size=150, window=8, min_count=5, sg=1, workers=10)
model.build_vocab(df)  # prepare the model vocabulary

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [81]:
#training the model
model.train(sentences=df, total_examples=len(df), epochs=model.iter)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(987705, 1478415)

In [83]:
#save the model
model.save('./data/model')

In [84]:
word_vectors = model.wv
count = 0
for word in word_vectors.vocab:
    if count<10:
        print(word)
        count += 1
    else:
        break

tambe
pero
diria
que
estar
vist
millor
quan
mort
buenas


In [85]:
len(word_vectors.vocab)

7696

In [86]:
print(model.wv.most_similar(positive=["mujer","rey"],negative=["hombre"]))

[('juegan', 0.897331953048706), ('salvo', 0.8857386112213135), ('pudo', 0.884837806224823), ('ganado', 0.8817203044891357), ('muerte', 0.8701326847076416), ('calle', 0.869951605796814), ('siendo', 0.8695622086524963), ('tendria', 0.8689562082290649), ('error', 0.8686603307723999), ('cambio', 0.8672849535942078)]


In [ ]:
#now, lets load da big boi model

from numpy import  zeros, float32 as REAL
from gensim.models import keyedvectors
import codecs

# this function was build using code excerpts from:
# https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/keyedvectors.py
def load_vectors_from_csv(fname,vocab_size=973265,vector_size=100):
    print("Loading vectors from file:",fname)
    result=keyedvectors.KeyedVectors(vector_size=100)
    result.syn0 = zeros((vocab_size, vector_size), dtype=REAL)
    result.vecor_size=vector_size
    counts=None   
    def add_word(word, weights):
        word_id = len(result.vocab)
        if word in result.vocab:
            print("duplicate word '%s' in %s, ignoring all but first", word, fname)
            return
        if counts is None:
            # most common scenario: no vocab file given. just make up some bogus counts, in descending order
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=vocab_size - word_id)
        elif word in counts:
            # use count from the vocab file
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=counts[word])
        else:
            # vocab file given, but word is missing -- set count to None (TODO: or raise?)
            print("vocabulary file is incomplete: '%s' is missing", word)
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=None)
        result.syn0[word_id] = weights
        result.index2word.append(word)   
    file=codecs.open(fname,"r","utf-8")
    i=0
    for line in file:
        i+=1
        if i==1: #ommit header
            continue
        parts=line.strip().split(",")
        word,weights=parts[1],[REAL(x) for x in parts[2:]]
        add_word(word,weights)
        if i%100000==0:
            print(i,"word vectors loaded so far ...")
    file.close()
    print(i-1,"word vectors loaded!")
    return result
    

    
model=load_vectors_from_csv("data/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos2.csv")